# Capítulo 04 - Trabalhando com Parquet

Este notebook explora o formato Parquet, um formato colunar otimizado para grandes volumes de dados, amplamente usado em análise de dados e data lakes.

## 📚 Tópicos Abordados:
1. Introdução ao Formato Parquet
2. Instalação e Preparação
3. Leitura de Arquivos Parquet
4. Exportação para Parquet
5. Compressão em Parquet
6. Particionamento de Dados
7. Schema e Metadados
8. Performance: Parquet vs CSV
9. Parquet com Múltiplos Arquivos
10. Conversão CSV → Parquet
11. Boas Práticas e Otimizações

## 1. Introdução ao Formato Parquet

### O que é Parquet?
- **Formato colunar**: Armazena dados por coluna, não por linha
- **Compressão eficiente**: Economiza espaço e aumenta velocidade
- **Otimizado para leitura**: Perfeito para análise de dados
- **Compatível**: Usado por Spark, Pandas, DuckDB, etc.

### Vantagens sobre CSV:
- ✅ **Menor tamanho** (compressão automática)
- ✅ **Leitura mais rápida** (formato colunar)
- ✅ **Preserva tipos de dados** (sem conversões)
- ✅ **Metadados incluídos** (schema embutido)
- ✅ **Suporta dados complexos** (nested structures)

## 2. Instalação e Preparação

In [ ]:
# Instalar dependências
%pip install duckdb pandas pyarrow -q
print("✓ Pacotes instalados!")

In [ ]:
import duckdb
import pandas as pd
import os

print(f"DuckDB versão: {duckdb.__version__}")
print(f"Pandas versão: {pd.__version__}")
print("\n✓ Imports realizados!")

## 3. Leitura de Arquivos Parquet

### 3.1 Criar Dados de Teste e Exportar como Parquet

In [ ]:
# Criar conexão
con = duckdb.connect()

# Criar dados de exemplo e salvar como Parquet
con.sql("""
    COPY (
        SELECT 
            i AS id,
            'Usuario_' || i AS nome,
            20 + (i % 50) AS idade,
            CASE (i % 5)
                WHEN 0 THEN 'São Paulo'
                WHEN 1 THEN 'Rio de Janeiro'
                WHEN 2 THEN 'Belo Horizonte'
                WHEN 3 THEN 'Brasília'
                ELSE 'Curitiba'
            END AS cidade,
            (i % 10 + 1) * 1000.0 AS salario,
            (i % 2 = 0) AS ativo
        FROM range(1, 1001) t(i)
    ) TO 'example.parquet'
""")

print("✓ Arquivo 'example.parquet' criado!")
print(f"Tamanho: {os.path.getsize('example.parquet'):,} bytes")

### 3.2 Ler com `read_parquet()`

In [ ]:
# Ler Parquet como Relation
rel = duckdb.read_parquet("example.parquet")

print("Tipo do objeto:", type(rel))
print("\nPrimeiras 5 linhas:")
rel.limit(5).show()

print("\n✓ Parquet lido com sucesso!")

### 3.3 Query SQL Direta

In [ ]:
# Query direta no arquivo Parquet
print("Consulta SQL direta:")
result = duckdb.sql("SELECT * FROM 'example.parquet' LIMIT 10")
result.show()

print("\n✓ Query executada!")

### 3.4 Filtros e Agregações

In [ ]:
# Filtros
print("Pessoas com salário > 7000:")
duckdb.sql("""
    SELECT nome, idade, cidade, salario
    FROM 'example.parquet'
    WHERE salario > 7000
    ORDER BY salario DESC
    LIMIT 10
""").show()

# Agregações por cidade
print("\nEstatísticas por cidade:")
duckdb.sql("""
    SELECT 
        cidade,
        COUNT(*) AS total,
        AVG(idade) AS idade_media,
        AVG(salario) AS salario_medio,
        SUM(CASE WHEN ativo THEN 1 ELSE 0 END) AS ativos
    FROM 'example.parquet'
    GROUP BY cidade
    ORDER BY total DESC
""").show()

### 3.5 Converter para DataFrame

In [ ]:
# Converter para Pandas
df = duckdb.read_parquet("example.parquet").df()

print("DataFrame criado:")
print(df.head())
print(f"\nShape: {df.shape}")
print(f"Colunas: {df.columns.tolist()}")
print(f"\nTipos de dados:")
print(df.dtypes)

## 4. Exportação para Parquet

### 4.1 Exportação Básica

In [ ]:
# Criar tabela temporária
con.sql("""
    CREATE OR REPLACE TABLE usuarios AS
    SELECT * FROM 'example.parquet' WHERE idade >= 30
""")

# Exportar usando write_parquet()
result = con.sql("SELECT * FROM usuarios")
result.write_parquet("usuarios_30plus.parquet")

print("✓ Arquivo 'usuarios_30plus.parquet' criado!")
print(f"Tamanho: {os.path.getsize('usuarios_30plus.parquet'):,} bytes")

# Verificar conteúdo
print("\nPrimeiras linhas do arquivo exportado:")
duckdb.sql("SELECT * FROM 'usuarios_30plus.parquet' LIMIT 5").show()

### 4.2 Exportação com COPY TO

In [ ]:
# Usar COPY TO (mais controle)
con.sql("""
    COPY (
        SELECT cidade, COUNT(*) AS total, AVG(salario) AS salario_medio
        FROM usuarios
        GROUP BY cidade
    ) TO 'resumo_cidades.parquet' (FORMAT parquet)
""")

print("✓ Arquivo 'resumo_cidades.parquet' criado!")

# Verificar
print("\nConteúdo:")
duckdb.sql("SELECT * FROM 'resumo_cidades.parquet'").show()

## 5. Compressão em Parquet

Parquet suporta diferentes algoritmos de compressão.

### 5.1 Algoritmos Disponíveis

In [ ]:
# Testar diferentes compressões
compressoes = ['uncompressed', 'snappy', 'gzip', 'zstd']

print("Comparação de compressões:\n")
print(f"{'Compressão':<15} {'Tamanho (bytes)':<20} {'Economia'}")
print("=" * 50)

for comp in compressoes:
    arquivo = f'test_{comp}.parquet'
    
    con.sql(f"""
        COPY (
            SELECT * FROM 'example.parquet'
        ) TO '{arquivo}' (FORMAT parquet, COMPRESSION {comp})
    """)
    
    tamanho = os.path.getsize(arquivo)
    
    if comp == 'uncompressed':
        tamanho_original = tamanho
        economia = "-"
    else:
        economia = f"{(1 - tamanho/tamanho_original)*100:.1f}%"
    
    print(f"{comp:<15} {tamanho:>15,}     {economia}")

print("\n✓ Comparação concluída!")

### 5.2 Recomendações de Compressão

- **SNAPPY**: Mais rápido, compressão moderada (padrão)
- **GZIP**: Compressão melhor, mais lento
- **ZSTD**: Melhor equilíbrio (recomendado)
- **Uncompressed**: Apenas para testes

## 6. Particionamento de Dados

Particionar dados melhora performance em grandes volumes.

### 6.1 Criar Dados Particionados

In [ ]:
# Criar diretório
os.makedirs('dados_particionados', exist_ok=True)

# Particionar por cidade
con.sql("""
    COPY (
        SELECT * FROM 'example.parquet'
    ) TO 'dados_particionados' 
    (FORMAT parquet, PARTITION_BY (cidade))
""")

print("✓ Dados particionados criados!\n")

# Listar partições
print("Partições criadas:")
for root, dirs, files in os.walk('dados_particionados'):
    for file in files:
        caminho = os.path.join(root, file)
        tamanho = os.path.getsize(caminho)
        print(f"  {caminho}: {tamanho:,} bytes")

### 6.2 Ler Dados Particionados

In [ ]:
# Ler todos os arquivos particionados
print("Lendo dados particionados:")
duckdb.sql("""
    SELECT cidade, COUNT(*) AS total
    FROM 'dados_particionados/**/*.parquet'
    GROUP BY cidade
    ORDER BY cidade
""").show()

print("\n✓ DuckDB lê automaticamente partições!")

### 6.3 Filtro de Partição (Partition Pruning)

In [ ]:
# Consulta que usa apenas uma partição
print("Consulta com filtro de partição:")
duckdb.sql("""
    SELECT nome, idade, salario
    FROM 'dados_particionados/**/*.parquet'
    WHERE cidade = 'São Paulo'
    LIMIT 5
""").show()

print("\n✓ DuckDB lê apenas a partição necessária!")

## 7. Schema e Metadados

### 7.1 Inspecionar Schema

In [ ]:
# Ver schema do arquivo Parquet
print("Schema do arquivo:")
duckdb.sql("DESCRIBE SELECT * FROM 'example.parquet'").show()

# Estatísticas das colunas
print("\nEstatísticas:")
duckdb.sql("""
    SELECT 
        COUNT(*) AS total_linhas,
        COUNT(DISTINCT cidade) AS cidades_distintas,
        MIN(idade) AS idade_min,
        MAX(idade) AS idade_max,
        MIN(salario) AS salario_min,
        MAX(salario) AS salario_max
    FROM 'example.parquet'
""").show()

### 7.2 Metadados do Parquet

In [ ]:
# Informações sobre o arquivo Parquet
print("Metadados do arquivo Parquet:")
duckdb.sql("""
    SELECT 
        file_name,
        row_group_id,
        row_group_num_rows,
        total_compressed_size
    FROM parquet_metadata('example.parquet')
    LIMIT 5
""").show()

print("\n✓ Metadados lidos!")

## 8. Performance: Parquet vs CSV

### 8.1 Criar Arquivos de Teste

In [ ]:
# Criar dataset grande
con.sql("""
    CREATE OR REPLACE TABLE dataset_grande AS
    SELECT 
        i AS id,
        'Nome_' || i AS nome,
        20 + (i % 60) AS idade,
        'Cidade_' || (i % 100) AS cidade,
        (i % 20 + 1) * 500.0 AS valor,
        DATE '2024-01-01' + INTERVAL (i % 365) DAY AS data,
        (i % 2 = 0) AS flag
    FROM range(1, 100001) t(i)
""")

# Exportar como CSV e Parquet
con.sql("COPY dataset_grande TO 'benchmark.csv' (HEADER)")
con.sql("COPY dataset_grande TO 'benchmark.parquet'")

# Comparar tamanhos
tamanho_csv = os.path.getsize('benchmark.csv')
tamanho_parquet = os.path.getsize('benchmark.parquet')

print("Comparação de tamanho (100k linhas):")
print(f"CSV:     {tamanho_csv:>10,} bytes ({tamanho_csv/1024/1024:.2f} MB)")
print(f"Parquet: {tamanho_parquet:>10,} bytes ({tamanho_parquet/1024/1024:.2f} MB)")
print(f"\nEconomia: {(1 - tamanho_parquet/tamanho_csv)*100:.1f}%")

### 8.2 Benchmark de Leitura

In [ ]:
import time

# Teste CSV
start = time.time()
result_csv = duckdb.sql("""
    SELECT cidade, COUNT(*), AVG(valor)
    FROM 'benchmark.csv'
    WHERE idade > 40
    GROUP BY cidade
""").fetchall()
time_csv = time.time() - start

# Teste Parquet
start = time.time()
result_parquet = duckdb.sql("""
    SELECT cidade, COUNT(*), AVG(valor)
    FROM 'benchmark.parquet'
    WHERE idade > 40
    GROUP BY cidade
""").fetchall()
time_parquet = time.time() - start

print("Benchmark: Agregação em 100k linhas")
print("=" * 45)
print(f"CSV:     {time_csv*1000:>8.2f}ms")
print(f"Parquet: {time_parquet*1000:>8.2f}ms")
print(f"\nParquet é {time_csv/time_parquet:.1f}x mais rápido!")

### 8.3 Benchmark de Leitura Seletiva (Projeção)

In [ ]:
# Ler apenas 2 colunas (formato colunar brilha aqui!)
print("Leitura seletiva (apenas 2 de 7 colunas):\n")

# CSV - precisa ler tudo
start = time.time()
result_csv = duckdb.sql("SELECT idade, valor FROM 'benchmark.csv'").fetchall()
time_csv = time.time() - start

# Parquet - lê apenas as colunas necessárias
start = time.time()
result_parquet = duckdb.sql("SELECT idade, valor FROM 'benchmark.parquet'").fetchall()
time_parquet = time.time() - start

print(f"CSV:     {time_csv*1000:>8.2f}ms")
print(f"Parquet: {time_parquet*1000:>8.2f}ms")
print(f"\nParquet é {time_csv/time_parquet:.1f}x mais rápido!")
print("\n💡 Formato colunar só lê as colunas necessárias!")

## 9. Parquet com Múltiplos Arquivos

### 9.1 Criar Múltiplos Arquivos

In [ ]:
# Criar diretório
os.makedirs('vendas_parquet', exist_ok=True)

# Criar arquivos para diferentes meses
meses = ['jan', 'fev', 'mar', 'abr']
for i, mes in enumerate(meses, 1):
    con.sql(f"""
        COPY (
            SELECT 
                j AS id,
                '{mes}' AS mes,
                'Produto_' || (j % 20) AS produto,
                (j % 100 + 10) * 5.0 AS valor
            FROM range({(i-1)*50 + 1}, {i*50 + 1}) t(j)
        ) TO 'vendas_parquet/vendas_{mes}.parquet'
    """)

print("✓ Arquivos Parquet criados:")
for arquivo in os.listdir('vendas_parquet'):
    tamanho = os.path.getsize(f'vendas_parquet/{arquivo}')
    print(f"  - {arquivo}: {tamanho:,} bytes")

### 9.2 Ler Todos os Arquivos

In [ ]:
# Glob pattern para ler múltiplos Parquet
print("Agregação em múltiplos arquivos:")
duckdb.sql("""
    SELECT 
        mes,
        COUNT(*) AS total_vendas,
        SUM(valor) AS valor_total,
        AVG(valor) AS valor_medio,
        MIN(valor) AS valor_min,
        MAX(valor) AS valor_max
    FROM 'vendas_parquet/*.parquet'
    GROUP BY mes
    ORDER BY mes
""").show()

print("\n✓ Todos os arquivos processados em uma query!")

## 10. Conversão CSV → Parquet

### 10.1 Conversão Simples

In [ ]:
# Criar CSV de exemplo
csv_data = """id,produto,preco,quantidade
1,Notebook,3500.00,5
2,Mouse,50.00,100
3,Teclado,150.00,50
4,Monitor,800.00,20
5,Webcam,200.00,30
"""

with open('produtos.csv', 'w') as f:
    f.write(csv_data)

# Converter para Parquet
duckdb.sql("""
    COPY (
        SELECT * FROM 'produtos.csv'
    ) TO 'produtos.parquet'
""")

# Comparar tamanhos
csv_size = os.path.getsize('produtos.csv')
parquet_size = os.path.getsize('produtos.parquet')

print("Conversão CSV → Parquet:")
print(f"CSV:     {csv_size:,} bytes")
print(f"Parquet: {parquet_size:,} bytes")
print(f"Economia: {(1 - parquet_size/csv_size)*100:.1f}%")

# Verificar dados
print("\nDados convertidos:")
duckdb.sql("SELECT * FROM 'produtos.parquet'").show()

### 10.2 Conversão em Lote

In [ ]:
# Criar múltiplos CSVs
os.makedirs('csv_origem', exist_ok=True)
os.makedirs('parquet_destino', exist_ok=True)

for i in range(1, 4):
    con.sql(f"""
        COPY (
            SELECT 
                j AS id,
                'Item_' || j AS nome,
                (j * 10.5) AS valor
            FROM range({(i-1)*20 + 1}, {i*20 + 1}) t(j)
        ) TO 'csv_origem/dados_{i}.csv' (HEADER)
    """)

print("✓ CSVs criados:")
for arquivo in os.listdir('csv_origem'):
    print(f"  - {arquivo}")

# Converter todos para Parquet (sem particionamento complexo)
print("\nConvertendo...")
duckdb.sql("""
    COPY (
        SELECT * FROM 'csv_origem/*.csv'
    ) TO 'parquet_destino/dados_convertidos.parquet' (FORMAT parquet)
""")

print("\n✓ Conversão concluída!")
print("\nArquivos Parquet criados:")
for root, dirs, files in os.walk('parquet_destino'):
    for file in files:
        caminho = os.path.join(root, file)
        tamanho = os.path.getsize(caminho)
        print(f"  - {caminho}: {tamanho:,} bytes")

## 11. Boas Práticas e Otimizações

### 📊 Dicas de Uso:

1. **Use Parquet para dados analíticos**: Ideal para data lakes e análise
2. **Particione dados grandes**: Melhora performance em queries filtradas
3. **Escolha compressão ZSTD**: Melhor equilíbrio velocidade/tamanho
4. **Aproveite formato colunar**: SELECT apenas colunas necessárias
5. **Evite muitos arquivos pequenos**: Combine em arquivos maiores
6. **Use para dados imutáveis**: Parquet não é para dados que mudam frequentemente

### 11.1 Exemplo Otimizado

In [ ]:
# ❌ Não Otimizado
# df = pd.read_csv('grande.csv')
# df_filtrado = df[df['valor'] > 1000][['id', 'nome', 'valor']]

# ✅ Otimizado com Parquet
df_otimizado = duckdb.sql("""
    SELECT id, nome, valor
    FROM 'benchmark.parquet'
    WHERE valor > 5000
    LIMIT 10
""").df()

print("Consulta otimizada:")
print(df_otimizado)

print("\n✓ Parquet + projeção de colunas + filtro = máxima eficiência!")

### 11.2 Quando Usar Parquet vs CSV

| Característica | CSV | Parquet |
|---------------|-----|--------|
| **Tamanho** | Grande | Pequeno (comprimido) |
| **Velocidade leitura** | Lento | Muito rápido |
| **Leitura seletiva** | Lê tudo | Lê apenas colunas necessárias |
| **Tipos de dados** | Texto (precisa converter) | Preserva tipos |
| **Compatibilidade** | Universal | Requer bibliotecas |
| **Edição manual** | Fácil (texto) | Impossível |
| **Uso recomendado** | Dados pequenos, interoperabilidade | Dados grandes, análise |

### Recomendação:
- 📄 **Use CSV** para: Dados pequenos, troca de dados, edição manual
- 📊 **Use Parquet** para: Data lakes, análise de grandes volumes, armazenamento eficiente

## 🎯 Resumo do Capítulo

Neste capítulo, exploramos:

1. ✅ **Introdução ao Parquet** - formato colunar otimizado
2. ✅ **Leitura** com `read_parquet()` e SQL direto
3. ✅ **Exportação** com múltiplas opções
4. ✅ **Compressão** (SNAPPY, GZIP, ZSTD)
5. ✅ **Particionamento** para melhor performance
6. ✅ **Schema e metadados** embutidos
7. ✅ **Performance** superior ao CSV
8. ✅ **Múltiplos arquivos** com glob patterns
9. ✅ **Conversão** CSV → Parquet
10. ✅ **Boas práticas** e otimizações

### 🔑 Pontos-Chave:
- Parquet é **muito mais rápido e compacto** que CSV
- Formato **colunar** permite leitura seletiva
- **Preserva tipos** de dados e metadados
- Ideal para **data lakes** e análise
- **Particionamento** melhora performance

### 📚 Próximo Capítulo:
Importação e Exportação de JSON!

## 🧹 Limpeza (Opcional)

In [ ]:
import shutil

# Arquivos
arquivos = [
    'example.parquet', 'usuarios_30plus.parquet', 'resumo_cidades.parquet',
    'test_uncompressed.parquet', 'test_snappy.parquet', 'test_gzip.parquet', 'test_zstd.parquet',
    'benchmark.csv', 'benchmark.parquet', 'produtos.csv', 'produtos.parquet'
]

for arquivo in arquivos:
    if os.path.exists(arquivo):
        os.remove(arquivo)
        print(f"✓ Removido: {arquivo}")

# Diretórios
diretorios = ['dados_particionados', 'vendas_parquet', 'csv_origem', 'parquet_destino']
for dir in diretorios:
    if os.path.exists(dir):
        shutil.rmtree(dir)
        print(f"✓ Diretório '{dir}' removido")

print("\n✓ Limpeza concluída!")